# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_key import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City Name,Latitude,Longitude,Max Temp,Humidity,Clouds,Wind,Country,Date
0,0,ust-nera,64.5667,143.2000,-11.18,84,90,2.47,RU,1729140531
1,1,adamstown,-25.0660,-130.1015,21.32,85,100,8.33,PN,1729140480
2,2,albany,42.6001,-73.9662,3.23,62,3,1.72,US,1729140468
3,3,iqaluit,63.7506,-68.5145,-0.15,100,100,6.71,CA,1729140534
4,4,bilibino,68.0546,166.4372,-2.83,96,100,1.71,RU,1729140536


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Longitude', 
    y='Latitude',   
    size='Humidity',  
    geo=True,
    tiles='OSM',  # OpenStreetMap
    title='City Humidity Map'
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 20
ideal_temp_max = 45
ideal_humidity_max = 90

ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) &
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Humidity'] < ideal_humidity_max)
]

# Drop any rows with null values
ideal_weather_df_clean = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_clean

,City_ID,City Name,Latitude,Longitude,Max Temp,Humidity,Clouds,Wind,Country,Date
1,1,adamstown,-25.0660,-130.1015,21.32,85,100,8.33,PN,1729140480
5,5,port mathurin,-19.6833,63.4167,23.21,79,89,5.51,MU,1729140537
6,6,mhamid,29.8200,-5.7200,22.43,49,99,2.46,MA,1729140538
7,7,nova sintra,14.8667,-24.7167,26.29,81,12,3.13,CV,1729140539
8,8,west island,-12.1568,96.8225,28.99,74,20,7.72,CC,1729140500


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City Name', 'Country', 'Latitude', 'Longitude', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City Name,Country,Latitude,Longitude,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,85,
5,port mathurin,MU,-19.6833,63.4167,79,
6,mhamid,MA,29.8200,-5.7200,49,
7,nova sintra,CV,14.8667,-24.7167,81,
8,west island,CC,-12.1568,96.8225,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [57]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "limit": 20,  # Limit to the first hotel found
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]  # Replace with the actual column name for latitude
    longitude = row['Longitude']  # Replace with the actual column name for longitude
    
    print(f"{longitude},{latitude} -LATLONGS")
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"]=f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    if response.status_code == 200:
        data = response.json()  # Convert the response to JSON format
    # Now you can process the JSON data as needed
        if data['features']:
            hotel_info = data['features'][0]['properties']
            name_address = {
                "name": hotel_info.get('name'),  # Get the hotel name
                "address": hotel_info.get('address_line2')  # Get the address
            }
            print(name_address)  # Display the hotel name and address
        else:
            print("No hotel found.")
    else:
        print("Error:", response.status_code)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as e:
        print(f"Key error: {e}")
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City Name']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
-130.1015,-25.066 -LATLONGS
No hotel found.
Key error: string indices must be integers
adamstown - nearest hotel: No hotel found
63.4167,-19.6833 -LATLONGS
{'name': 'Escale Vacances', 'address': 'Johnston street, Port Mathurin, Mauritius'}
Key error: 'features'
port mathurin - nearest hotel: No hotel found
-5.72,29.82 -LATLONGS
{'name': 'Hotel Kasbah Azalay', 'address': "RN9, 47404 M'Hamid El Ghizlane, Morocco"}
Key error: 'features'
mhamid - nearest hotel: No hotel found
-24.7167,14.8667 -LATLONGS
{'name': 'Residência Ka Dencho', 'address': 'Nova Sintra -Nossa Senhora do Monte - Cachaço, 9110-Cova Rodela, Cape Verde'}
Key error: 'features'
nova sintra - nearest hotel: No hotel found
96.8225,-12.1568 -LATLONGS
{'name': 'Cocos Village Bungalows', 'address': 'Clunies Ross Avenue, West Island 6799, Australia'}
Key error: 'features'
west island - nearest hotel: No hotel found


,City Name,Country,Latitude,Longitude,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,85,No hotel found
5,port mathurin,MU,-19.6833,63.4167,79,No hotel found
6,mhamid,MA,29.8200,-5.7200,49,No hotel found
7,nova sintra,CV,14.8667,-24.7167,81,No hotel found
8,west island,CC,-12.1568,96.8225,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Latitude",
    "Longitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Latitude,Longitude]   (Hotel Name,Country)